## IGDBenchmark


In [1]:
#from google.colab import drive
#drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!git clone https://github.com/vohoaidanh/IGDBenchmark.git

Cloning into 'IGDBenchmark'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 190 (delta 95), reused 153 (delta 67), pack-reused 0
Receiving objects: 100% (190/190), 3.46 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [2]:
!apt-get install -y unzip
!apt-get install -y zip
!pip install imageio -q
!pip install opencv-python -q
!apt-get install -y libgl1-mesa-glx -q

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 63 not upgraded.
Need to get 175 kB of archives.
After this operation, 386 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2 [175 kB]
Fetched 175 kB in 0s (1890 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package unzip.
(Reading database ... 22469 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
Unpacking unzip (6.0-26ubuntu3.2) ...
Setting up unzip (6.0-26ubuntu3.2) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and

In [12]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 120.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 68.7 MB/s eta 0:00:00


In [14]:
!pip install scikit-image -q

In [4]:
!pip install ftfy -q
!pip install natsort -q
!pip install tensorboardX -q
!pip install blobfile -q
#!pip install mpi4py -q
#!pip install grad-cam -q
!pip install comet_ml -q

In [5]:
import os
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory:", cwd)

Current working directory: /mywork


In [6]:
!unzip -q /mywork/RealFakeDB512s_PIE.zip -d /mywork/rgb

In [7]:
!unzip -q /mywork/RealFakeDB512s.zip -d /mywork/shading

In [8]:
!mv /mywork/RealFakeDB512s/mywork/PIE-Net/results/PIE/RealFakeDB512s /mywork/IGDBenchmark

In [9]:
!cd /mywork/IGDBenchmark

In [15]:
# @title Training parameters
# @markdown Type to this form.

gpu_ids = 0  # @param {type: "number"}
dataroot = './mydataset/'  # @param {type: "string"}
checkpoints_dir = './Shading_and_RGB'  # @param {type: "string"}
batch_size = 64  # @param {type: "number"}
niter = 50  # @param {type: "number"}
num_threads = 4 # @param {type: "number"}
save_epoch_freq = 5  # @param {type: "number"}
detect_method = 'shading'  # @param {type: "string"}

# @markdown Set [epoch_start] if [--continue_train] is used.
epoch_start = 0  # @param {type: "number"}
# @markdown ---


In [ ]:
!python train.py --name Shading_and_RGB --blur_prob 0.1 --blur_sig 0.0,3.0 --jpg_prob 0.1 --jpg_method cv2,pil \
--jpg_qual 30,100 \
--dataroot $dataroot \
--checkpoints_dir $checkpoints_dir \
--detect_method $detect_method \
--gpu_ids $gpu_ids \
--num_threads $num_threads \
--batch_size $batch_size \
--lr 0.0001 \
--train_split train \
--val_split val \
--niter $niter \
--save_epoch_freq $save_epoch_freq \
--comet

----------------- Options ---------------
                     arch: res50                         
               batch_size: 64                            	[default: 1]
                    beta1: 0.9                           
                blur_prob: 0.1                           	[default: 0]
                 blur_sig: 0.0,3.0                       	[default: 0.5]
          checkpoints_dir: ./Shading_and_RGB             	[default: ./checkpoints]
                class_bal: False                         
                  classes:                               
                    comet: True                          	[default: False]
           continue_train: False                         
                 cropSize: 224                           
                 data_aug: False                         
                 dataroot: ./mydataset/                  	[default: ./mydataset/ShadingDB/]
            detect_method: shading                       	[default: CNNDetection]
     

In [51]:
# Run evaluation script. Model weights need to be downloaded. See eval_config.py for flags
!python eval.py --no_crop --batch_size 1

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).
COMET INFO: Experiment is live on comet.com https://www.comet.com/danhvohoai2-gmail-com/ai-generated-image-detection/5ff28514c7b444a9ab525a31ec9eeb79

CNNDetection_model_epoch_best model testing on...
(test) acc: 0.9758; TPR: 0.9642, TNR: 0.9874
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : managing_bat_8356
COMET INFO:     url                   : https://www.comet.com/danhvohoai2-gmail-com/ai-generated-image-detection/5ff28514c7b444a9ab525a31ec9eeb79
COMET INFO:   Metrics:
COMET INFO:     corsstest/acc : 0.9758
COMET INFO:   Parameters:
COMET INFO:     Cross_test params : {"CropSize": 22

In [27]:
import os
ims = []
for a,b,c in os.walk('/mywork/IGDBenchmark/mydataset/shading'):
    if len(c)>0:
        ims.extend([os.path.join(a,i) for i in c])
    

In [30]:
len(ims)

56000

In [26]:
for i in ims:
    file_name = os.path.basename(i)
    new_name = i.replace('_shading','')
    os.rename(i, new_name)
